In [1]:
import numpy as np
import cv2
import dlib
import pickle
from imutils import face_utils

In [32]:
with open('classifier.pickle', 'rb') as f:
    classifier = pickle.load(f)
with open('mean.pickle', 'rb') as f:
    m = pickle.load(f)
with open('std.pickle', 'rb') as f:
    std = pickle.load(f)

In [43]:
# Directory of the pretrained model
p = "shape_predictor_68_face_landmarks.dat"

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(p)

image = cv2.imread('demo.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Get faces into webcam's image
rects = detector(gray, 0)

for rect in rects:
    # Make the prediction and transfom it to numpy array
    shape = predictor(gray, rect)
    landmarks = face_utils.shape_to_np(shape)

In [44]:
# 3D model points.
model_points = np.array([
                            (0.0, 0.0, 0.0),             # Nose tip
                            (0.0, -330.0, -65.0),        # Chin
                            (-225.0, 170.0, -135.0),     # Left eye left corner
                            (225.0, 170.0, -135.0),      # Right eye right corne
                            (-150.0, -150.0, -125.0),    # Left Mouth corner
                            (150.0, -150.0, -125.0)      # Right mouth corner
                        ])
size = image.shape
focal_length = size[1]
center = (size[1]/2, size[0]/2)
camera_matrix = np.array(
                         [[focal_length, 0, center[0]],
                         [0, focal_length, center[1]],
                         [0, 0, 1]], dtype = "double"
                         )
dist_coeffs = np.zeros((4,1)) # Assuming no lens distortion

# Grab the 2D coordinates of our six sample points
image_points = np.array([
    (landmarks[33][0], landmarks[33][1]) ,     # Nose tip
    (landmarks[8][0], landmarks[8][1]),     # Chin
    (landmarks[36][0], landmarks[36][1]),     # Left eye left corner
    (landmarks[45][0], landmarks[45][1]),     # Right eye right corner
    (landmarks[48][0], landmarks[48][1]),     # Left Mouth corner
    (landmarks[54][0], landmarks[54][1])      # Right mouth corner
], dtype="double")

In [45]:
# Solve the PnP problem with the parameters specified above
# and obtain rotation and translation vectors
(success, rotation_vector, translation_vector) = cv2.solvePnP(
    model_points, image_points, camera_matrix, dist_coeffs, flags=cv2.SOLVEPNP_ITERATIVE
    )

In [50]:
x = np.concatenate((rotation_vector, translation_vector), axis=0)
x = x.squeeze()
x = (x-m)/std
x = x.reshape(1, -1)
print(x)

[[-0.30568279 -0.04698829 -0.36194454  0.3299144   0.40853588 -0.91300547]]


In [48]:
x = np.array([-0.3950592, 0.00481095, 0.10784691, 0.73003738, 1.6652573 , -0.16816019])
x = x.reshape(1, -1)
y_hat = classifier.predict(x)
print(y_hat)

[1.]
